In [ ]:
### Load Packages

import pyodbc
import pandas as pd
import numpy as np
import pickle

In [ ]:
### Open Python Format of SDH Taxonomy

with open('SDOH_codes_complete.p', 'rb') as fp:
    SDOH_code_map = pickle.load(fp)

In [ ]:
### Create ICD-10 Codes Table

icd10_codes_1 = {}

for key in SDOH_code_map.keys():
    icd10_codes_1[key] = (SDOH_code_map[key]['icd10'])

icd10_codes_2 = {}
for k,v in icd10_codes_1.items():
    for x in v:
        icd10_codes_2.setdefault(x,[]).append(k)
        
icd10table = pd.DataFrame(icd10_codes_2.items(), columns=['icd10', 'SDOH'])

In [ ]:
### Clean-Up Table

icd10table.icd10 = icd10table.icd10.astype(str)
icd10table.icd10 = icd10table.icd10.str.upper()

icd10table['SDOH'] = icd10table['SDOH'].astype(str).replace("\['", '', regex=True)
icd10table['SDOH'] = icd10table['SDOH'].astype(str).replace("\']", '', regex=True)

icd10table.head(3)

In [ ]:
## Create Codes List

icd10s = list(icd10table['icd10'])

In [ ]:
### Connect to Database and Run SQL Query

dsn_name = "DSN=DBS_4_Python"
conn = pyodbc.connect(dsn_name)
cur = conn.cursor()

sql_deid = """
SELECT * FROM dbs.schema.diagnosis_table WHERE UPPER(icd10) in {} ;
""".format('('+str(icd10s)[1:-1]+')')

icd10_data = pd.read_sql(sql_deid, conn)

### Note, this is just one example of connecting to a ODBC database system and pulling its data directly into 
### a python environment. Please review documentation for the pyodbc package at 
### https://github.com/mkleehammer/pyodbc and contact your DBMS administrator for instructions on how to access 
### your specific database system

In [ ]:
### Attached SDH Lables to the Extracted Data

labled_icd10_data = pd.merge(icd10_data, icd10table, how='left',on=['icd10'])

In [11]:
### Alternatively, you can save your python dataframe as a csv and upload it to your DBMS as a table. You can then run SQL 
## querys and JOINS to extract and label SDH data.

icd10table.to_csv("icd10_table.csv")

### SQL Example

SELECT * FROM
    (SELECT * FROM dbs.schema.diagnosis_table WHERE UPPER(icd10) IN 
        (SELECT icd10 FROM sandbox.schema.icd10_table)) as boo
    LEFT JOIN  (SELECT * FROM sandbox.schema.icd10_table) as coo
    ON boo.icd10 = coo.icd10